In [1]:
#黑客入侵Linux服务器后，会进行提权操作，获取root权限。
#通过收集Linux服务器的Bash操作日志，这里仅仅给出了操作命令，而非完整命令。对特定用户的操作习惯进行训练，然后识别黑客的异常操作

# 数据收集和数据清洗
训练数据有50个用户的操作日志，每个日志有15000条操作命令，前5000条是正常操作，后10000条中存在攻击操作。我们只选择了一个user3来进行机器学习训练。

# 特征化
1. 对日志中所有操作命令使用次数进行排序，选出使用最多的前50条记录做`M1`，选出使用最少的前50条记做`M2`。
2. 每100条操作命令组成一个操作序列，这样的序列有150个，这100条操作中不重复的命令个数，记做`F1`。这100条操作中最多的10条且在`M1`中的命令个数（重合度），记做`F2`。这100条操作中最少的10条且在`M2`中的命令个数（重合度），记做`F3`。一个序列`{F1,F2,F3}`作为这个序列的特征，这样的特征有150个，前面110个作为训练集，后面40个作为测试集。
3. 打标签，前50个正常全部是0，后100个有0也有1。这150个标签，对应到第二步中的150个特征。前面110个作为训练，后面40个作为测试。

> 这个特征是否对应特定用户有效？物理意义是什么？

# 训练模型
使用KNN函数进行训练。

In [2]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x_train, y_train)
y_predict = neigh.predict(x_test)

NameError: name 'KNeighborsClassifier' is not defined

In [3]:
#完整代码

In [23]:
import sys
import urllib
#import urlparse
import re
import numpy as np
from sklearn.externals import joblib
#import HTMLParser
import nltk
import csv
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn import metrics

#测试样本数
N = 110

def load_user_cmd(filename):
    cmd_list = []
    dist_max = []
    dist_min = []
    dist = []
    with open(filename) as f:
        i = 0
        x = []
        for line in f:
            #print(line)
            line = line.strip('\n')
            x.append(line)
            dist.append(line)
            i += 1
            if i == 100:
                cmd_list.append(x)
                x = []
                i = 0
                
    fdist = FreqDist(dist).most_common()
    dist_max = set([i[0] for i in fdist[0:50]])
    dist_min = set([j[0] for j in fdist[-50:0]])
    return cmd_list, dist_max,dist_min

def get_user_cmd_feature(user_cmd_list, dist_max, dist_min):
    user_cmd_feature = []
    for cmd_block in user_cmd_list:
        f1 = len(set(cmd_block))
        fdist = FreqDist(cmd_block).most_common()
        f2 = [i[0] for i in fdist[0:10]]
        f3 = [j[0] for j in fdist[-10:]]
        f2 = len(set(f2)&set(dist_max))
        f3 = len(set(f3)&set(dist_min))
        x = [f1, f2, f3]
        user_cmd_feature.append(x)
    return user_cmd_feature

def get_label(filename, index = 0):
    x = []
    with open(filename) as f:
        #print(f.readlines())
        for line in f.readlines():
            line = line.strip('\n')
            #print('长度', line)
            x.append(int(line.split()[index]))
            
    return x

if __name__ == '__main__':
    user_cmd_list, user_cmd_dist_max, user_cmd_dist_min = load_user_cmd('./data/MasqueradeDat/User10')
    user_cmd_feature = get_user_cmd_feature(user_cmd_list, user_cmd_dist_max, user_cmd_dist_min)
    
    labels = get_label('./data/MasqueradeDat/label.txt', 2)
    
    y = [0] * 50 + labels
    x_train = user_cmd_feature[0:N]
    y_train = y[0:N]
    
    x_test = user_cmd_feature[N:150]
    y_test = y[N:150]
    
    neigh = KNeighborsClassifier(n_neighbors = 3)
    neigh.fit(x_train, y_train)
    y_predict = neigh.predict(x_test)
    
    score = np.mean(y_test==y_predict)*100
    print('score:', score)
    print(classification_report(y_test, y_predict))

score: 100.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        40

avg / total       1.00      1.00      1.00        40

